Flask pour rest API et jsonpickle pour code la reponse avant de l'envoyer à l'application

In [ ]:
!pip install flask-ngrok
!pip install jsonpickle

dans le cas des pancarte on utilise un detecteur de text

In [ ]:
!pip install psenet-text-detector

importer les packages nécessaire

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils
import math
from skimage.filters import threshold_sauvola, threshold_local
from sklearn.externals import joblib 
from sklearn.preprocessing import StandardScaler
import imutils
from sklearn.neural_network import MLPClassifier
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from flask_ngrok import run_with_ngrok
from flask import Flask, request, Response
import jsonpickle

connexion au Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

pour les models on utilise CNN pour les doc imprimées et SVM pour les pancarte

In [ ]:
clf=tf.keras.models.load_model('/content/drive/My Drive/PFE licence_Groupe3/CNN3.h5') # doc imprimés
clf2=joblib.load( '/content/drive/My Drive/Models/svm(norm).pkl')# pancartes

importer les packages nécessaire

In [ ]:
import psenet_text_detector as psenet
from flask import Flask, request, Response
from flask_ngrok import run_with_ngrok
import io
import base64 
import numpy as np
from PIL import Image
import jsonpickle
import numpy as np
import cv2
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [25,12]
from skimage.filters import threshold_sauvola
from sklearn.externals import joblib 
from sklearn.preprocessing import StandardScaler
import imutils
from sklearn.neural_network import MLPClassifier
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import psenet_text_detector as psenet
psenet_model = psenet.load_psenet_model()

PSENet text detector weight will be downloaded to /root/psenet_text_detector/weights/psenet_best.pth


Downloading...
From: https://drive.google.com/uc?id=1w2IgRkQXX49AbOARitO5xCr8-N93JHDd
To: /root/psenet_text_detector/weights/psenet_best.pth
229MB [00:02, 102MB/s]


In [ ]:
import psenet_text_detector as psenet

Prétraitement pour les pancarte

In [ ]:
def binarize(img):
    tresh = threshold_sauvola(img.copy(), window_size=25)
    
    return ((img > tresh)*255).astype(np.uint8)
def otsu(img):
    return cv2.threshold(img, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
def denoise2(img):
    return cv2.medianBlur(img.copy(),5)
def correcto(txt):
  txt=txt.lower()
  newtxt=list(txt)
  for i in range(len(newtxt)):

    if (newtxt[i]=='o'):
      if(newtxt[i-1].isnumeric()):
        newtxt[i]='0'

    if (newtxt[i]=='0'):
      if(not(newtxt[i-1].isnumeric())):
        newtxt[i]='o'

  newtxt="".join(newtxt)
  return newtxt

**segmentation**, **extraction de caracteristiques** et **normalisation** de l'image pour **les pancartes**

In [ ]:
def same_ligne(a,b):
  if(abs(a[1]-b[1])<65):
    return True
  return False
def list_c(boxes):
  #fing image center
  nb_boxes=boxes.shape[0]
  list_C=np.zeros((nb_boxes,2))
  for i in range(nb_boxes):
    pts=boxes[i,:,:]
    x1,y1 = pts.min(axis=0)
    x2,y2 = pts.max(axis=0)
    x_center=int((x1+x2)/2)
    y_center=int((y1+y2)/2)
    center=(x_center,y_center)
    list_C[i,:]=center

  return list_C
def count_lig(vect):
  #count words on the same line
  k=1
  ct=[]
  for i in range(vect.shape[0]-1):
    a=vect[i]
    b=vect[i+1]
    if(same_ligne(a,b)):
      k+=1
      continue
    ct.append(k)
    k=1 
  ct.append(k)
  return ct
def sort_all(list_C,boxes):
  #sort words 
  k=0
  new_boxes=np.zeros((boxes.shape[0],4,2)).astype(int)
  ct_r=count_lig(list_C)
  for i in range(len(ct_r)):
    tmp=ct_r[i]
    if(tmp==1):
      new_boxes[k,:,:]=boxes[k,:,:]
      k+=1
    else:
      tmp_boxes=boxes[k:k+tmp,:,:]
      sort_ind=np.argsort(list_C[k:k+tmp,0])
      # for j in range(len(sort_ind)):
      new_boxes[k:k+tmp,:,:]=tmp_boxes[sort_ind,:,:]
      k=k+tmp

  return new_boxes

def correct(cont,img):
    #perspective correction
    lig=cont.shape[0]
    cont=cont.reshape(lig,2)
#     print(lig)
    
    rect = np.zeros((4,2), dtype="float32")
    
    
    #estimation du quadrilatère
    s = np.sum(cont, axis=1)
    rect[0] = cont[np.argmin(s)]
    rect[2] = cont[np.argmax(s)]
    
    diff = np.diff(cont, axis=1)
    rect[1] = cont[np.argmin(diff)]
    rect[3] = cont[np.argmax(diff)]
    

    (A, B, C, D) = rect
    
    
    #max(hauteur,largeur) du quadrilatère 
    widthA = np.sqrt((A[0] - B[0])**2 + (A[1] - B[1])**2 )
    widthB = np.sqrt((D[0] - C[0])**2 + (D[1] - C[1])**2 )
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt((A[0] - D[0])**2 + (A[1] - D[1])**2 )
    heightB = np.sqrt((B[0] - C[0])**2 + (B[1] - C[1])**2 )
    maxHeight = max(int(heightA), int(heightB))
    
    #quadrilatère de reference
    dst = np.array([
    [0,0],
    [maxWidth-1, 0],
    [maxWidth-1, maxHeight-1],
    [0, maxHeight-1]], dtype="float32")
    
    #matrice de transformation(quad d'origine --> quad de reference)
    BansformMaBix = cv2.getPerspectiveTransform(rect, dst)
    #transformation affine
    # scan = cv2.warpPerspective(img.copy(), BansformMaBix, (maxWidth, maxHeight),borderMode=cv2.BORDER_REPLICATE)
    
    return cv2.warpPerspective(img.copy(), BansformMaBix, (maxWidth, maxHeight),borderMode=cv2.BORDER_REPLICATE)
def pretseg(c):
  #find the main line of the box 
  kernel = np.ones((1,180), np.uint8)
  p = cv2.dilate(255-c, kernel, iterations=1)
  _,contours, hierarchy = cv2.findContours(p, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  area=[]
  for cnt in contours: 
      rect = cv2.minAreaRect(cnt)
      area.append(rect[1][0]*rect[1][1])
      box = cv2.boxPoints(rect)
      box = np.int0(box)
  m=max(enumerate(area),key=lambda x: x[1])[0]
  # cnt = max(cnts , key = cv2.contourArea)
  rect = cv2.minAreaRect(contours[m])
  box = cv2.boxPoints(rect)
  box = np.int0(box)
  if rect[2]==0:
      return c
  else:
      return correct(box,c)   
def normalisa(img):

  pts = np.argwhere(img==0)
  y1,x1 = pts.min(axis=0)
  y2,x2 = pts.max(axis=0)
  img = img.copy()[y1:y2+1, x1:x2+1]
  img=cv2.resize(img,(40,80),interpolation=cv2.INTER_NEAREST)
  [l,c]=img.shape
  A=np.ones([128,128],dtype='uint8')*255
  espH=128-c
  espV=128-l
  espH=espH//2
  espV=espV//2
  A[espV:espV+l,espH:espH+c]=img

        
  return A


def caractere(n):
    c='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
    return c[n]
def findlimit(bin1):
  #find the start and the stop pixels of a caractere
  bin1=(bin1/255).T
  l,c=bin1.shape
  b=np.ones([l,c])
  h=c-np.sum(bin1,axis=1)
  start=[]
  stop=[]
  for i in range(l-1):
      if h[i]==0 and h[i+1]!=0 :
          start.append(i+1)
      else:
          if h[i]!=0 and h[i+1]==0 :
              stop.append(i+2)
  return start,stop



**Extraction de caracteristiques**

In [ ]:
def extractHOG(size, frame):
    winSize = size
    blockSize = (16,16)
    blockStride = (8,8)
    cellSize = (16,16)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
    if frame is None:
        return None
    return hog.compute(frame)
def caract(img):
    hog=extractHOG((128,128),img)
    hog=hog.T
    hog=hog.ravel()
    
    return hog

**Classification**

In [ ]:
def fcatR(txt,b,s,st):
  #cut caracters of a word
  for i in range(len(s)):
    if ((b[:,s[i]:st[i]].size-float(cv2.countNonZero(b[:,s[i]:st[i]])))/(b[:,s[i]:st[i]].shape[0]*b[:,s[i]:st[i]].shape[1]))>0.1:
      cropped = b[:,s[i]:st[i]]
      vect=caract(normalisa(cropped))
      vect= np.array(vect).reshape(-1,1)
      car=clf2.predict(vect.T)
      txt=txt+caractere(car[0])   
  return txt

decodage de l'image 
image arrivée est codée en **base64** donc il faut le decoder

In [ ]:
def stringToImage(base64_string):
    imgdata = base64.b64decode(base64_string)
    return Image.open(io.BytesIO(imgdata))
var=np.zeros([20,20])
# convert PIL Image to an RGB image( technically a numpy array ) that's compatible with opencv
def toRGB(image):
    return cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)
    
def correctRot(ind,img):
  if(ind==6):
    img=cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
  elif(ind==8):
    img=cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
  elif(ind==3):
    img=cv2.rotate(img, cv2.ROTATE_180)
  return img


**pretraitement** doc imprimée

In [ ]:
def readimg(path):
    #converts rgb ro grayscale
    img = cv2.imread(path,0)
    return img
def denoise(img):
    blur = cv2.GaussianBlur(img.copy(),(5,5),0)
    return blur
def denoise2(img):
    median = cv2.medianBlur(img.copy(),5)
    return median
def binarize(img):
    tresh = threshold_sauvola(img.copy(), window_size=25)
    bin1 = (img > tresh)*255
    bin1=bin1.astype(np.uint8)
    return bin1
def otsu(img):
    bin1=cv2.threshold(img, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    return bin1
def canny(img):
    edged = cv2.Canny(img.copy(), 75, 200)
    return edged

def detect_ctrs(img):
    
    blur=denoise(img.copy())
    blur=denoise(blur)
    ed1=canny(blur)
    ed1=canny(ed1)
    
    return ed1
def largest_cont(edg):
    
    _,cnts, hierarchy  = cv2.findContours(edg.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    #selectionner le contour ayant la plus grande surface
    cnt = max(cnts , key = cv2.contourArea)
    
    
    #approximation du perimetre(True --> contour fermé)
    peri = cv2.arcLength(cnt, True)
    
    #approcimation du polygone
    approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
    return approx

def cont_area(edg):
    _,cnts, hierarchy  = cv2.findContours(edg.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    cnt = max(cnts , key = cv2.contourArea)
    
    return cv2.contourArea(cnt)
def correct(cont,img):
    
    lig=cont.shape[0]
    cont=cont.reshape(lig,2)
#     print(lig)
    
    rect = np.zeros((4,2), dtype="float32")
    
    
    #estimation du quadrilatère
    s = np.sum(cont, axis=1)
    rect[0] = cont[np.argmin(s)]
    rect[2] = cont[np.argmax(s)]
    
    diff = np.diff(cont, axis=1)
    rect[1] = cont[np.argmin(diff)]
    rect[3] = cont[np.argmax(diff)]
    

    (A, B, C, D) = rect
    
    
    #max(hauteur,largeur) du quadrilatère 
    widthA = np.sqrt((A[0] - B[0])**2 + (A[1] - B[1])**2 )
    widthB = np.sqrt((D[0] - C[0])**2 + (D[1] - C[1])**2 )
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt((A[0] - D[0])**2 + (A[1] - D[1])**2 )
    heightB = np.sqrt((B[0] - C[0])**2 + (B[1] - C[1])**2 )
    maxHeight = max(int(heightA), int(heightB))
    
    #quadrilatère de reference
    dst = np.array([
    [0,0],
    [maxWidth-1, 0],
    [maxWidth-1, maxHeight-1],
    [0, maxHeight-1]], dtype="float32")
    
    #matrice de transformation(quad d'origine --> quad de reference)
    BansformMaBix = cv2.getPerspectiveTransform(rect, dst)
    #transformation affine
    scan = cv2.warpPerspective(img.copy(), BansformMaBix, (maxWidth, maxHeight),borderMode=cv2.BORDER_REPLICATE)
    
    return scan


**Segmentation** et normalisation pour les doc imprimées

In [ ]:
def text_only(img):
    ed=detect_ctrs(img)
    pts = np.argwhere(ed>0)
    y1,x1 = pts.min(axis=0)
    y2,x2 = pts.max(axis=0)
    cropped = img.copy()[y1:y2, x1:x2]    
    return cropped

def pretseg(c):
    kernel = np.ones((5,100), np.uint8)
    p = cv2.dilate(255-c, kernel, iterations=1)
    _,contours, hierarchy = cv2.findContours(p, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    area=[]
    im=c.copy()
    for cnt in contours: 
        rect = cv2.minAreaRect(cnt)
        area.append(rect[1][0]*rect[1][1])
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        im = cv2.drawContours(im,[box],0,(0,0,255),2)
    m=max(enumerate(area),key=lambda x: x[1])[0]
    rect = cv2.minAreaRect(contours[m])
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    if rect[2]==0:
        return c
    else:
        return correct(box,c)      

def findlimit(bin1):
    bin1=bin1/255
    bin1=bin1.T
    l,c=bin1.shape
    b=np.ones([l,c])
    h=np.sum(bin1,axis=1)
    h=c-h
    start=[]
    stop=[]
    for i in range(l-1):
        if h[i]==0 and h[i+1]!=0 :
            start.append(i+1)
        else:
            if h[i]!=0 and h[i+1]==0 :
                stop.append(i+2)
    return start,stop 

def normalisa(img):
    if (0 in img):
        pts = np.argwhere(img==0)
        y1,x1 = pts.min(axis=0)
        y2,x2 = pts.max(axis=0)
        img = img.copy()[y1:y2+1, x1:x2+1]

        img=cv2.resize(img,(40,80),interpolation=cv2.INTER_NEAREST)

        [l,c]=img.shape
        A=np.ones([128,128],dtype='uint8')*255


        espH=128-c
        espV=128-l
        espH=espH//2
        espV=espV//2
        A[espV:espV+l,espH:espH+c]=img

        img=A

        
    else:
        A=np.ones([128,128],dtype='uint8')*255
        
    return A



def text_split(thresh):
    im=thresh.copy()
    #dilation
    kernel = np.ones((5,100), np.uint8)
    img_dilation = cv2.dilate(255-thresh, kernel, iterations=1)
    l,c=im.shape
    #find contours
    _,ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #sort contours
    #contours = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    contours=np.flip(ctrs,axis=0)
    mask = np.zeros(im.shape, dtype=np.uint8)
    txt=''
    for idx in range(len(contours)):
        x, y, w, h = cv2.boundingRect(contours[idx])
        # Getting ROI
        roi = im[y:y+h, x:x+w]
        cv2.drawContours(mask, contours, idx,255, -1)
        # show ROI
        r = float(cv2.countNonZero(mask[y:y+h, x:x+w])) / (w * h)
        if r > 0.3 and w > 200 and h > 20 and h<0.4*l :
            p=pretseg(roi)
            a,b=findlimit(p)
            ########################################## fcatR ou fcat
            txt=fcat(txt,p,a,b) 
    return txt



def draw_cont(img,edg):
    _,cnts, hierarchy  = cv2.findContours(edg.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    cnt = max(cnts , key = cv2.contourArea)
    
    return cv2.drawContours(img.copy(), [cnt], -1, (0,255,0), 3)

**extraction de caractéristique**

In [ ]:
def extractHOG(size, frame):
    winSize = size
    blockSize = (16,16)
    blockStride = (8,8)
    cellSize = (16,16)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
    if frame is None:
        return None
    return hog.compute(frame)
def caract(img):
    vect=[]
    hog=extractHOG((128,128),img)
    hog=hog.T
    hog=hog.ravel()
    vect=np.hstack((vect,hog.T))
    
    return vect

**classification**

In [ ]:
def fcat(txt,b,s,st):
    k=1
    space=[]
    for i in range(len(s)-1):
        space.append(s[i+1]-st[i])
    if len(space)==0:
      return txt
    else:
      m=sum(space)/len(space)
    #print(m)
    for i in range(len(s)-1):
        cropped = b[:,s[i]:st[i]]
        if (np.where(cropped==0)[0].size/cropped.size)>0.08 and 4<cropped.shape[1]<150 :
            vect=caract(normalisa(cropped))
            vect=vect.reshape(1,45,45,1)
            car=clf.predict(vect)
            txt=txt+caractere(np.argmax(car))
            #print(txt)
            k=k+1 
        if s[i+1]-st[i]>m+8 :
        #if s[i+1]-st[i]>20 :
          #print(s[i+1]-st[i])
          txt=txt+' '
          k=k+1
    cropped = b[:,s[-1]:st[-1]]
    vect=caract(normalisa(cropped))
    vect=vect.reshape(1,45,45,1)
    car=clf.predict(vect)
    txt=txt+caractere(np.argmax(car))
    k=k+1
    txt=txt+' '
    return txt
def caractere(n):
    c=['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    #c='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
    return c[n]

REST API 

apres execution de cette cellule ci-dessus, colab va generer un lien qui va lier l'application mobile et notre API

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)
@app.route('/api/doc', methods=['POST'])
def ocr():
  #r = request
  # convert string of image data to uint8
  #nparr = np.fromstring(r.data, np.uint8)
  # decode image
  #img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
  r = request.get_json()
  r=r['dt']
  r=r['_parts']
  r=r[0]
  r=r[1]
  r=str(r['uri'])
  ind=int(r[0])
  b64=r[1:]
  img=stringToImage(b64)
  # print(r[0])
  # img=cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
  img=toRGB(img)
  img=correctRot(ind,img)
  # img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  rgb=img
  img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  length=img.shape[0]

  if(length>4000):
      val=7.5
  elif(length>1500):
      val=5.5
  elif(length>870):
      val=4
  else :
      val=1

  div=int(length/val)

  im2 = imutils.resize(img, height = div)
  rgb=imutils.resize(rgb, height = div)
  ratio = img.shape[0] / div
  vs=im2.copy()
  ed=detect_ctrs(vs)
  paper=largest_cont(ed)
  # paper=largest_cont(paper,ed)
  if(len(paper)==0):
      res=img
  else :
      area=cont_area(ed)
      draw=draw_cont(rgb,ed)

      #print('area :',area)
      if(area>400):
          tmp=correct(paper*ratio,img)
          co=1
          res=tmp
  #         res=denoise2(res)
          bin1=binarize(res)
          black=np.argwhere(bin1==0).size/2
          white=np.argwhere(bin1==255).size/2
          #print("back :",black/bin1.size)
          #print('white :',white/bin1.size)
          #print("size :",tmp.size/img.size)
          if((black!=0 and black/bin1.size>0.86) or (white!=0 and white/bin1.size>0.995) or (tmp.size/img.size)<0.02):
              tmp=img
      else:
          tmp=img

      res=tmp
  #     res=denoise2(res)

  bin1=binarize(res)

  #plt.subplot(1,3,1),plt.imshow(cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB))
  #if(len(paper)!=0):
  #    plt.subplot(1,3,2),plt.imshow(cv2.cvtColor(draw, cv2.COLOR_BGR2RGB))
  #plt.subplot(1,3,1),plt.imshow(bin1,'gray',interpolation='bicubic')
  #plt.show()

  #bin1=otsu(tmp)
  l,c=bin1.shape
  nbin=np.ones([l+2,c+2])*255
  nbin[1:-1,1:-1]=bin1
  nbin = nbin.astype(np.uint8)
  txt = text_split(nbin)
  response = { 'text':txt}
                
    # encode response using jsonpickle
  response_pickled = jsonpickle.encode(response)

  return Response(response=response_pickled, status=200, mimetype="application/json")
 



  ##################################### test appMobile

@app.route('/api/pancarte', methods=['POST'])
def ocr2():
  ##################################### test appMobile
  r = request.get_json()
  r=r['dt']
  r=r['_parts']
  r=r[0]
  r=r[1]
  r=str(r['uri'])
  ind=int(r[0])
  b64=r[1:]
  img=stringToImage(b64)
  img=toRGB(img)
  img=correctRot(ind,img)
  ######################################## test clien
  #r = request
  #nparr = np.fromstring(r.data, np.uint8)
  #img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
  ######################################################
  img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# prediction_result = psenet.detect_text(image_path, output_dir, cuda=False)

  prediction_result = psenet.get_prediction(image=img,
                               		  model=psenet_model,
                                       	  binary_th=1.0,
                                       	  kernel_num=3,
                                       	  upsample_scale=1,
                                       	  long_size=1280,
                                       	  min_kernel_area=10.0,
                                       	  min_area=100.0,
                                       	  min_score=0.93,
                                       	  )
  boxes=prediction_result["boxes"]
  list_C=list_c(boxes)
  new_boxes=sort_all(list_C,boxes)
  image=img
  nb_boxes=boxes.shape[0]
  txt=''
  for i in range(nb_boxes):
    pts=new_boxes[i,:,:]
    x1,y1 = pts.min(axis=0)
    x2,y2 = pts.max(axis=0)
    if(x1==0):
      x1=2
    if(y1==0):
      y1=2
    gray = cv2.cvtColor(image[y1-2:y2+2,x1-2:x2+2], cv2.COLOR_BGR2GRAY)

    # sharpen image
    sharpen = cv2.GaussianBlur(gray, (0,0), 3)
    sharpen = cv2.addWeighted(gray, 1.5, sharpen, -0.5, 0)

    # apply adaptive threshold to get black and white effect
    thresh = otsu(sharpen)
    #crop=binarize(gray)

    #tranform all sort of image to an image with white backg
    _,ctr,_=cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cntsSorted = sorted(ctr, key=lambda x: cv2.contourArea(x))
    x, y,_,_ = cv2.boundingRect(cntsSorted[-1])
    if x!=0 and y!=0:
      thresh=255-thresh
    #crop=thresh
    
    crop=pretseg(denoise2(denoise2(thresh)))
    nbin=np.ones([crop.shape[0]+2,crop.shape[1]+2])*255
    nbin[1:-1,1:-1]=crop
    nbin = nbin.astype(np.uint8)

    #find caracters's rectangle with height>image's height
    a=np.zeros([1,4])
    _,ctr,_=cv2.findContours(255-nbin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours = sorted(ctr, key=lambda ctrs: cv2.boundingRect(ctrs)[0])
    for idx in range(len(contours)):
      if cv2.boundingRect(contours[idx])[3]>nbin.shape[0]*0.3 :
        a=np.vstack((a,np.array([cv2.boundingRect(contours[idx])[0],cv2.boundingRect(contours[idx])[1],cv2.boundingRect(contours[idx])[2],cv2.boundingRect(contours[idx])[3]])))
    a=a[1:,:].astype(np.uint32)

    #erase rectangles intersection 
    for r in range (a.shape[0]-1):
      if a[r,0]+a[r,2]>= a[r+1,0]:
        nbin[:,a[r+1,0]:a[r,0]+a[r,2]+1]=255
    
    s,st=findlimit(nbin)
    txt=fcatR(txt,nbin,s,st)
    txt=txt+' '
    if i==0:
      ddd=nbin

  txt=txt.lower()
  txt=correcto(txt)
  response = {'text':txt}
  response_pickled = jsonpickle.encode(response)

  return Response(response=response_pickled, status=200, mimetype="application/json")

app.run()
##https://reactnativecode.com/upload-image-to-server-using-php-mysql/
app.run() 